In [6]:
import pandas as pd
import numpy as np

import plotly.express as px
import plotly.graph_objects as go

import matplotlib.pyplot as plt
import seaborn as sns

from fluid_properties.regression import DataLoader, train

In [7]:
CONFIG = {
    # Data Parameters
    "FLUID": "CO2",
    "data_path": "data/CO2_nist_data.csv",
    "output_dir": "outputs",

    # Model parameters
    "num_splits": 50,
    "max_poly_degree": 3,
    "random_state": 42,
    "test_size": 0.2,

    # Features
    "x_feature": "Temperature (C)",
    "y_feature": "Pressure (bar)",
    "z_feature": "Density (kg/m3)",
    "error_feature": "Absolute Error, Density (kg/m3)"
}

In [41]:
loader = DataLoader(CONFIG)
predictor, df_errors, df_coeffecients = train(loader, CONFIG)
predictor.fit(loader.X_train, loader.Y_train)

df_coeffecients.mean().to_csv(
    f'{CONFIG["output_dir"]}/{CONFIG["FLUID"]}_coefficients.csv', index_label='Feature Name', header=['Coeffecient']
)
df_errors.to_csv(
    f'{CONFIG["output_dir"]}/{CONFIG["FLUID"]}_regression_errors.csv', index_label='Split Number'
)

df_test = pd.DataFrame()
df_test[CONFIG['x_feature']], df_test[CONFIG['y_feature']] = loader.X_test.T
df_test[CONFIG['z_feature']] = loader.Y_test
df_test[f'Predicted {CONFIG["z_feature"]}'] = predictor.predict(
    loader.X_test)
df_test[CONFIG["error_feature"]] = (
    df_test[f'Predicted {CONFIG["z_feature"]}'] - df_test[CONFIG["z_feature"]]).abs()

Existing data found: c:\Users\josep\OneDrive\Documents\GitHub\nel/data/CO2_nist_data.csv


Training model with polynomial degree 3: 100%|██████████| 50/50 [00:01<00:00, 33.36split/s]


Best model with polynomial degree: 3


In [52]:
fig = go.Figure(data=[
    go.Mesh3d(
        x=df_test[CONFIG['x_feature']],
        y=df_test[CONFIG['y_feature']],
        z=df_test[CONFIG['z_feature']],
        colorscale='Viridis',
        intensity=df_test[CONFIG["error_feature"]],
    )]
)
fig.show()